In [2]:
using BitemporalPostgres
using LifeInsuranceDataModel, LifeInsuranceProduct
using SearchLight
using SearchLightPostgreSQL
using TimeZones
ENV["SEARCHLIGHT_USERNAME"] = "mf"
ENV["SEARCHLIGHT_PASSWORD"] = "mf"
SearchLight.Configuration.load() |> SearchLight.connect

SearchLight.query("DROP SCHEMA public CASCADE")
SearchLight.query("CREATE SCHEMA public")
LifeInsuranceDataModel.load_model()

┌ Info: DROP SCHEMA public CASCADE
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  drop cascades to 37 other objects
DETAIL:  drop cascades to table schema_migrations
drop cascades to table histories
drop cascades to table versions
drop cascades to table workflows
drop cascades to table testtstzranges
drop cascades to table validityintervals
drop cascades to table testdummycomponents
drop cascades to table testdummycomponentrevisions
drop cascades to table testdummysubcomponents
drop cascades to table testdummysubcomponentrevisions
drop cascades to extension btree_gist
drop cascades to function f_bitempranges()
drop cascades to function f_versionrange()
drop cascades to table contractpartnerroles
drop cascades to table tariffitempartnerroles
drop cascades to table tariffitemroles
drop cascades to table productpartroles
drop cascades to table contracts
drop cascades to table contractrevisions
drop cascades

┌ Info: already connected
└ @ LifeInsuranceDataModel /home/mf/dev/julia_depot/packages/LifeInsuranceDataModel/5ovLP/src/LifeInsuranceDataModel.jl:573
┌ Info: CREATE TABLE schema_migrations (version varchar(30))
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: Created table schema_migrations
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:306
┌ Info: CREATE TABLE histories (id BIGSERIAL PRIMARY KEY, dummy INTEGER )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE versions (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE workflows (id BIGSERIAL PRIMARY KEY, ref_history B

┌ Info: CREATE TABLE testdummyComponents (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE testdummyComponentRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES testdummyComponents(id), ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT DEFAULT 2^53 - 1 REFERENCES versions(id) ON DELETE SET DEFAULT, ref_valid INT8RANGE , description VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE testdummySubComponents (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES testdummyComponents(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_super BIGINT REFERENCES his

┌ Info: CREATE OR REPLACE FUNCTION f_bitempranges ()
│ RETURNS trigger AS
│ $$
│      DECLARE
│ 
│      BEGIN
│           RAISE NOTICE 'NEW: %', NEW;
│           NEW.tsrworld := tstzrange(NEW.tsworld_validfrom,NEW.tsworld_invalidfrom,'[)');
│           NEW.tsrdb := tstzrange(NEW.tsdb_validfrom,NEW.tsdb_invalidfrom,'[)');
│           RETURN NEW;
│      END;
│ $$ LANGUAGE 'plpgsql';
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TRIGGER versions_trig
│ BEFORE INSERT OR UPDATE ON validityIntervals
│ FOR EACH ROW EXECUTE PROCEDURE f_bitempranges();
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE OR REPLACE FUNCTION f_versionrange ()
│ RETURNS trigger AS
│ $$
│      DECLARE
│ 
│      BEGIN
│           RAISE NOTICE 'NEW: %', NEW;
│           NEW.ref_valid := int8range(NEW.ref_validfrom,NEW.ref_invalidfrom,'[)');
│    

┌ Info: CREATE TABLE contractRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES contracts(id) ON DELETE CASCADE, ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT DEFAULT 2^53 - 1 REFERENCES versions(id) ON DELETE SET DEFAULT, ref_valid INT8RANGE , description VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TRIGGER cr_versions_trig
│ BEFORE INSERT OR UPDATE ON contractRevisions
│ FOR EACH ROW EXECUTE PROCEDURE f_versionrange();
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: ALTER TABLE contractRevisions
│ ADD CONSTRAINT contractsversionrange EXCLUDE USING GIST (ref_component WITH =, ref_valid WITH &&)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE tariffs (id BI

...

┌ Info: INSERT INTO tariffitemroles ( "domain", "value" ) VALUES ( E'TariffItem', E'Main Coverage - Life' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffitemroles"."id" AS "tariffitemroles_id", "tariffitemroles"."domain" AS "tariffitemroles_domain", "tariffitemroles"."value" AS "tariffitemroles_value" FROM "tariffitemroles" WHERE "id" = 1 ORDER BY tariffitemroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: INSERT INTO tariffitemroles ( "domain", "value" ) VALUES ( E'TariffItem', E'Supplementary Coverage - Occupational Disablity' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffitemroles"."id" AS "tariffitemroles_id", "tariffitemroles"."domain" AS "tariffitemroles_domain", "tariffitemroles"."value" AS "tariffitemroles_value" FROM "tariffitemroles" WHERE "id" = 2 ORDER BY tariffitemroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO tariffitemroles ( "domain", "value" ) VALUES ( E'TariffItem', E'Supplementary Coverage - Terminal Illness' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffitemroles"."id" AS "tariffitemroles_id", "tariffitemroles"."domain" AS "tariffitemr

┌ Info: INSERT INTO productpartroles ( "domain", "value" ) VALUES ( E'ProductPart', E'Main Coverage - Life' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "productpartroles"."id" AS "productpartroles_id", "productpartroles"."domain" AS "productpartroles_domain", "productpartroles"."value" AS "productpartroles_value" FROM "productpartroles" WHERE "id" = 1 ORDER BY productpartroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: INSERT INTO productpartroles ( "domain", "value" ) VALUES ( E'ProductPart', E'Supplementary Coverage - Occupational Disablity' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "productpartroles"."id" AS "productpartroles_id", "productpartroles"."domain" AS "productpartroles_domain", "productpartroles"."value" AS "productpartroles_value" FROM "productpartroles" WHERE "id" = 2 ORDER BY productpartroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO productpartroles ( "domain", "value" ) VALUES ( E'ProductPart', E'Supplementary Coverage - Terminal Illness' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "productpartroles"."id" AS "productpartroles_id", "productpartroles"."domain" 

1×4 Matrix{ProductPartRole}:
 ProductPartRole
| KEY            | VALUE                |
|----------------|----------------------|
| domain::String | ProductPart          |
| id::DbId       | 1                    |
| value::String  | Main Coverage - Life |
  …  ProductPartRole
| KEY            | VALUE                |
|----------------|----------------------|
| domain::String | ProductPart          |
| id::DbId       | 4                    |
| value::String  | Profit participation |


In [3]:
using LifeInsuranceDataModel, LifeInsuranceProduct, SearchLight
import SearchLight: Serializer.serialize

tif = get_tariff_interface(Val(1))
tif.parameters


LifeRiskTariff = create_tariff("Life Risk Insurance", 1, "", serialize(tif.parameters))
ProfitParticipationTariff = create_tariff(
    "Profit participation", 2,
    "", "{}"
)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 1 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 1 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed", "type_of_entity" ) VALUES ( E'1', E'1', E'2038-01-19T03:14:06.999+00:00', E'2014-05-30T21:00:01.001+00:00', 0, E'Tariff' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "workflows_ref_history", "workflows"."ref_version" AS "workflows_ref_version", "workflows"."tsdb_validfrom" AS "workflows_tsdb_validfrom", "workflows"."tsw_validfrom" AS "workflows_tsw_validfrom", "workflows"."is_committed" AS "workflows_is_committed", "workflows"."type_of_entity" AS "workflows_type_of_entity" FROM "workflows" WHERE "id" = 1 ORDER BY workflows.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: INSERT INTO validityintervals ( "ref_history", "ref_version", "tsdb_validfrom", "tsworld_validfrom", "tsdb_invalidfrom", "tsworld_invalidfrom", "is_committed" ) VALUES ( E'1', E'1', E'2023-03-26T16:00:36.629+00:00', E'2014-05-30T21:00:01.001+00:00', E'2038-01-19T03:14:06.999+00:00', E'2038-01-19T03:14:06.999+00:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  NEW: (1,1,1,"2014-05-30 21:00:01.001+00","2038-01-19 03:14:06.999+00","2023-03-26 16:00:36.629+00","2038-01-19 03:14:06.999+00",,,0)
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_inv

...

┌ Info: INSERT INTO tariffpartnerrolerevisions ( "ref_component", "ref_validfrom", "ref_invalidfrom", "ref_role" ) VALUES ( E'1', E'1', E'9007199254740991', E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  NEW: (1,1,1,9007199254740991,,1)
┌ Info: SELECT "tariffpartnerrolerevisions"."id" AS "tariffpartnerrolerevisions_id", "tariffpartnerrolerevisions"."ref_component" AS "tariffpartnerrolerevisions_ref_component", "tariffpartnerrolerevisions"."ref_validfrom" AS "tariffpartnerrolerevisions_ref_validfrom", "tariffpartnerrolerevisions"."ref_invalidfrom" AS "tariffpartnerrolerevisions_ref_invalidfrom", "tariffpartnerrolerevisions"."ref_role" AS "tariffpartnerrolerevisions_ref_role" FROM "tariffpartnerrolerevisions" WHERE "id" = 1 ORDER BY tariffpartnerrolerevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'1' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgre

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history = E'1' AND tsrdb @> TIMESTAMPTZ E'2038-01-18T03:14:06.999+00:00' AND tsrworld && tstzrange(E'2014-05-31T21:00:01.001+00:00',E'2038-01-19T03:14:06.999+00:00') AND is_committed=1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: UPDATE validity

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'2' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgre

2

In [4]:
cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end

p = Product()
pr = ProductRevision(description="Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(
    ref_tariff=LifeRiskTariff,
    ref_role=ppRole["Main Coverage - Life"],
    description="Main Coverage - Life",
)

pp2 = ProductPart()
ppr2 = ProductPartRevision(
    ref_tariff=ProfitParticipationTariff,
    ref_role=ppRole["Profit participation"],
    description="Profit participation Life Risk",
)

w0 = Workflow(
    type_of_entity="Product",
    tsw_validfrom=ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
commit_workflow!(w0)

┌ Info: SELECT "contractpartnerroles"."id" AS "contractpartnerroles_id", "contractpartnerroles"."domain" AS "contractpartnerroles_domain", "contractpartnerroles"."value" AS "contractpartnerroles_value" FROM "contractpartnerroles" ORDER BY contractpartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffitempartnerroles"."id" AS "tariffitempartnerroles_id", "tariffitempartnerroles"."domain" AS "tariffitempartnerroles_domain", "tariffitempartnerroles"."value" AS "tariffitempartnerroles_value" FROM "tariffitempartnerroles" ORDER BY tariffitempartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: SELECT "tariffitemroles"."id" AS "tariffitemroles_id", "tariffitemroles"."domain" AS "tariffitemroles_domain", "tariffitemroles"."value" AS "tariffitemroles_value" FROM "tariffitemroles" ORDER BY tariffitemroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "productpartroles"."id" AS "productpartroles_id", "productpartroles"."domain" AS "productpartroles_domain", "productpartroles"."value" AS "productpartroles_value" FROM "productpartroles" ORDER BY productpartroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 3 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'3' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 3 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/p

┌ Info: INSERT INTO products ( "ref_history", "ref_version" ) VALUES ( E'3', E'3' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "products"."id" AS "products_id", "products"."ref_history" AS "products_ref_history", "products"."ref_version" AS "products_ref_version" FROM "products" WHERE "id" = 1 ORDER BY products.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: INSERT INTO productrevisions ( "ref_component", "ref_validfrom", "ref_invalidfrom", "description" ) VALUES ( E'1', E'3', E'9007199254740991', E'Life Risk' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  NEW: (1,1,3,9007199254740991,,"Life Risk")
┌ Info: SELECT "productrevisions"."id" AS "productrevisions_id", "productrevisions"."ref_component" AS "productrevisions_ref_component", "productrevisions"."ref_validfrom" AS "productrevisions_ref_validfrom", "productrevisions"."ref_invalidfrom" AS "productrevisions_ref_invalidfrom", "productrevisions"."description" AS "productrevisions_description" FROM "productrevisions" WHERE "id" = 1 ORDER BY productrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: INSERT INTO productparts ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'3', E'9007199254740991', E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "productparts"."id" AS "productparts_id", "productparts"."ref_history" AS "productparts_ref_history", "productparts"."ref_version" AS "productparts_ref_version", "productparts"."ref_super" AS "productparts_ref_super" FROM "productparts" WHERE "id" = 1 ORDER BY productparts.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: INSERT INTO productpartrevisions ( "ref_component", "ref_validfrom", "ref_invalidfrom", "ref_tariff", "ref_role", "description" ) VALUES ( E'1', E'3', E'9007199254740991', E'1', E'1', E'Main Coverage - Life' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  NEW: (1,1,,1,1,3,9007199254740991,,"Main Coverage - Life")
┌ Info: SELECT "productpartrevisions"."id" AS "productpartrevisions_id", "productpartrevisions"."ref_component" AS "productpartrevisions_ref_component", "productpartrevisions"."ref_validfrom" AS "productpartrevisions_ref_validfrom", "productpartrevisions"."ref_invalidfrom" AS "productpartrevisions_ref_invalidfrom", "productpartrevisions"."ref_tariff" AS "productpartrevisions_ref_tariff", "productpartrevisions"."ref_role" AS "productpartrevisions_ref_role", "productpartrevisions"."description" AS "productpartrevisions_description" FROM "productpartrevisions" WHERE "id" = 1 O

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO productparts ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'3', E'9007199254740991', E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "productparts"."id" AS "productparts_id", "productparts"."ref_history" AS "productparts_ref_history", "productparts"."ref_version" AS "productparts_ref_version", "productparts"."ref_super" AS "productparts_ref_super" FROM "productparts" WHERE "id" = 2 ORDER BY productparts.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO productpartrevisions ( "ref_componen

In [5]:
prs = prsection(1, now(tz"UTC"), ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"), 0)

┌ Info: already connected
└ @ LifeInsuranceDataModel /home/mf/dev/julia_depot/packages/LifeInsuranceDataModel/5ovLP/src/LifeInsuranceDataModel.jl:573
┌ Info: SELECT "products"."id" AS "products_id", "products"."ref_history" AS "products_ref_history", "products"."ref_version" AS "products_ref_version" FROM "products" WHERE id=E'1' ORDER BY products.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history=E'3' and tsrworld @> TIMESTAMPTZ E'2023-04-01T21:00:01.001+00:00' AND tsrdb @> TIMESTAMPTZ E'2023-03-26T16:00:50.652+00:00' AND is_committed = 1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "products"."id" AS "products_id", "produ

┌ Info: SELECT "productrevisions"."id" AS "productrevisions_id", "productrevisions"."ref_component" AS "productrevisions_ref_component", "productrevisions"."ref_validfrom" AS "productrevisions_ref_validfrom", "productrevisions"."ref_invalidfrom" AS "productrevisions_ref_invalidfrom", "productrevisions"."description" AS "productrevisions_description" FROM "productrevisions" WHERE ref_component=E'1' and ref_valid @> BIGINT E'3' ORDER BY productrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "productparts"."id" AS "productparts_id", "productparts"."ref_history" AS "productparts_ref_history", "productparts"."ref_version" AS "productparts_ref_version", "productparts"."ref_super" AS "productparts_ref_super" FROM "productparts" WHERE ref_history = BIGINT E'3' ORDER BY productparts.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLight

┌ Info: SELECT "productpartrevisions"."id" AS "productpartrevisions_id", "productpartrevisions"."ref_component" AS "productpartrevisions_ref_component", "productpartrevisions"."ref_validfrom" AS "productpartrevisions_ref_validfrom", "productpartrevisions"."ref_invalidfrom" AS "productpartrevisions_ref_invalidfrom", "productpartrevisions"."ref_tariff" AS "productpartrevisions_ref_tariff", "productpartrevisions"."ref_role" AS "productpartrevisions_ref_role", "productpartrevisions"."description" AS "productpartrevisions_description" FROM "productpartrevisions" WHERE ref_component=E'1' and ref_valid @> BIGINT E'3' ORDER BY productpartrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: already connected
└ @ LifeInsuranceDataModel /home/mf/dev/julia_depot/packages/LifeInsuranceDataModel/5ovLP/src/LifeInsuranceDataModel.jl:573
┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS 

┌ Info: SELECT "tariffpartnerroles"."id" AS "tariffpartnerroles_id", "tariffpartnerroles"."ref_history" AS "tariffpartnerroles_ref_history", "tariffpartnerroles"."ref_version" AS "tariffpartnerroles_ref_version", "tariffpartnerroles"."ref_super" AS "tariffpartnerroles_ref_super" FROM "tariffpartnerroles" WHERE ref_super=E'1' ORDER BY tariffpartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffpartnerrolerevisions"."id" AS "tariffpartnerrolerevisions_id", "tariffpartnerrolerevisions"."ref_component" AS "tariffpartnerrolerevisions_ref_component", "tariffpartnerrolerevisions"."ref_validfrom" AS "tariffpartnerrolerevisions_ref_validfrom", "tariffpartnerrolerevisions"."ref_invalidfrom" AS "tariffpartnerrolerevisions_ref_invalidfrom", "tariffpartnerrolerevisions"."ref_role" AS "tariffpartnerrolerevisions_ref_role" FROM "tariffpartnerrolerevisions" WHERE ref_component=E'1' and ref_vali

┌ Info: SELECT "productpartrevisions"."id" AS "productpartrevisions_id", "productpartrevisions"."ref_component" AS "productpartrevisions_ref_component", "productpartrevisions"."ref_validfrom" AS "productpartrevisions_ref_validfrom", "productpartrevisions"."ref_invalidfrom" AS "productpartrevisions_ref_invalidfrom", "productpartrevisions"."ref_tariff" AS "productpartrevisions_ref_tariff", "productpartrevisions"."ref_role" AS "productpartrevisions_ref_role", "productpartrevisions"."description" AS "productpartrevisions_description" FROM "productpartrevisions" WHERE ref_component=E'2' and ref_valid @> BIGINT E'3' ORDER BY productpartrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: already connected
└ @ LifeInsuranceDataModel /home/mf/dev/julia_depot/packages/LifeInsuranceDataModel/5ovLP/src/LifeInsuranceDataModel.jl:573
┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS 

ProductSection(ZonedDateTime(2023, 3, 26, 16, 0, 52, 89, tz"UTC"), ZonedDateTime(2023, 3, 26, 16, 0, 52, 89, tz"UTC"), 9007199254740991, 9007199254740991, ProductRevision
| KEY                   | VALUE            |
|-----------------------|------------------|
| description::String   | Life Risk        |
| id::DbId              | 1                |
| ref_component::DbId   | 1                |
| ref_invalidfrom::DbId | 9007199254740991 |
| ref_validfrom::DbId   | 3                |
, ProductPartSection[ProductPartSection(ProductPartRevision
| KEY                   | VALUE                |
|-----------------------|----------------------|
| description::String   | Main Coverage - Life |
| id::DbId              | 1                    |
| ref_component::DbId   | 1                    |
| ref_invalidfrom::DbId | 9007199254740991     |
| ref_role::DbId        | 1                    |
| ref_tariff::DbId      | 1                    |
| ref_validfrom::DbId   | 3                    |
, TariffSecti

In [6]:

# create partner 1
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1")
w = Workflow(type_of_entity="Partner",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)
Partner1 = p.id

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 4 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'4' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 4 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/p

┌ Info: INSERT INTO partners ( "ref_history", "ref_version" ) VALUES ( E'4', E'4' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "partners"."id" AS "partners_id", "partners"."ref_history" AS "partners_ref_history", "partners"."ref_version" AS "partners_ref_version" FROM "partners" WHERE "id" = 1 ORDER BY partners.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: INSERT INTO partnerrevisions ( "ref_component", "ref_validfrom", "ref_invalidfrom", "description", "date_of_birth" ) VALUES ( E'1', E'4', E'9007199254740991', E'Partner 1', E'2000-01-01' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  NEW: (1,1,4,9007199254740991,,"Partner 1",2000-01-01)
┌ Info: SELECT "partnerrevisions"."id" AS "partnerrevisions_id", "partnerrevisions"."ref_component" AS "partnerrevisions_ref_component", "partnerrevisions"."ref_validfrom" AS "partnerrevisions_ref_validfrom", "partnerrevisions"."ref_invalidfrom" AS "partnerrevisions_ref_invalidfrom", "partnerrevisions"."description" AS "partnerrevisions_description", "partnerrevisions"."date_of_birth" AS "partnerrevisions_date_of_birth" FROM "partnerrevisions" WHERE "id" = 1 ORDER BY partnerrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgre

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'4' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgre

1

In [ ]:
w1 = Workflow(
    type_of_entity="Contract",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)

create_entity!(w1)
c = Contract()
cr = ContractRevision(description="contract creation properties")
create_component!(c, cr, w1)

cpr = ContractPartnerRef(ref_super=c.id)
cprr = ContractPartnerRefRevision(
    ref_partner=Partner1,
    ref_role=cpRole["Policy Holder"],
    description="policiyholder ref properties",
)
create_subcomponent!(c, cpr, cprr, w1)
# pi 1
LifeRiskProduct = find(Product, SQLWhereExpression("id=?", 1))[1].id.value
PartnerRole = tiprRole["Insured Person"]

cpi = ProductItem(ref_super=c¨.id)
cpir = ProductItemRevision(
    ref_product=1,
    description="from contract creation",
)
create_subcomponent!(c, cpi, cpir, w1)

LifeInsuranceDataModel.create_product_instance(
    w1,
    cpi,
    LifeRiskProduct,
    Partner1.value,
    PartnerRole,
)
commit_workflow!(w1)

In [ ]:
cid = 1
h = find(Contract, SQLWhereExpression("id =?", cid))[1].ref_history
vi = find(ValidityInterval, SQLWhereExpression("ref_history=?", h), order=["ValidityInterval.id"])[1];
txntime = vi.tsdb_validfrom
reftime = vi.tsworld_validfrom

cs = csection(cid, txntime, reftime)

ti = cs.product_items[1].tariff_items[1]


In [69]:
using JSON, LifeInsuranceProduct
tariffparms = JSON.parse(ti.tariff_ref.rev.parameters)
mts = tariffparms["mortality_tables"]
interface_id = ti.tariff_ref.ref.revision.interface_id
tgt = get_tariff_interface(Val(interface_id)).calls["calculation_target"]
tgt["sex"] = "f"
tgt["smoker"] = "nonsmoker"
tgt["selected"] = "ä"
parms = tgt[tgt["selected"]]
parms["m"] = 10
parms["n"] = 20
parms["frequency"] = 2
parms["begin"] = string(Date("2023-04-01"))

parms
dob = ti.partner_refs[1].ref.revision.date_of_birth
issue_age = LifeInsuranceProduct.insurance_age(dob, Date(parms["begin"]))


life = SingleLife(
    mortality=MortalityTables.table(mts[tgt["sex"]][tgt["smoker"]]).select[issue_age])

yield = Yields.Constant(0.0125)      # Using a flat

lc = LifeContingency(life,yield)  # LifeContingenc


r1 = ä(lc)
r2 = ä(lc, 1)
r3 = ä(lc, 2)
r4 = ä(lc, 30)

println("r1")
println(r1)
println("r2")
println(r2)
println("r3")
println(r3)
println("r4")
println(r4)


MethodError: MethodError: no method matching LifeContingency(::LifeContingency{SingleLife{OffsetArrays.OffsetVector{Float64, Vector{Float64}}, Uniform}, Yields.Constant{Rate{Float64, Periodic}}})
Closest candidates are:
  LifeContingency(::L, !Matched::Y) where {L, Y} at ~/dev/julia_depot/packages/LifeContingencies/Bdhsr/src/LifeContingencies.jl:175
  LifeContingency(!Matched::I) where I<:Insurance at ~/dev/julia_depot/packages/LifeContingencies/Bdhsr/src/LifeContingencies.jl:280

In [ ]:

println("r1")
println(r1)
println("r2")
println(r2)
println("r3")
println(r3)
println("r4")
println(r4)
calculate!(ti)

println(ti.tariff_ref.rev)

In [62]:
using Dates, LifeContingencies
using MortalityTables
using Yields
import LifeContingencies: V, ä     # pull the shortform notation into scope

# load mortality rates from MortalityTables.jl
vbt2001 = MortalityTables.table("2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")

issue_age = 30

life = SingleLife(                 # The life underlying the risk
    mortality=vbt2001.select[issue_age],    # -- Mortality rates
)

yield = Yields.Constant(0.0125)      # Using a flat 1,25% interest rate

lc = LifeContingency(life, yield)  # LifeContingency joins the risk with interest


r1 = ä(lc)


36.61266492009452